# Hyperparameters optimization with keras-tuner

This example shows how to optimize the parameters of a neural network using keras-tuner. The problem is to predict gas flow based on the temperature and time of the day.   

Requirements for keras-tuner installation:

    Python 3.6
    TensorFlow 2.0
    
website: https://keras-team.github.io/keras-tuner/

In [1]:
#!pip install -U tensorflow

In [5]:
#!pip install -U keras-tuner

In [6]:
import tensorflow.python.keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras import backend as K

from tensorflow.python.keras import utils, layers, Sequential

import kerastuner
from kerastuner import RandomSearch
from kerastuner import BayesianOptimization

In [7]:
import pandas as pd
import numpy as np
import os

In [8]:
def load_data():
    """
    Read and prepare input data.

    The data will be split as follow:
        • X the training and testing instances of the dataset.
        • Y the label for the dataset instances.

    :return: four numpy arrays
    """

    from sklearn.model_selection import train_test_split
    df = pd.read_csv("file2.csv", sep=',')
    train, test = train_test_split(df, test_size=0.2)
    train_y = train['PUN_n']
    train_x = train.drop(['PUN_n'], axis=1)
    test_y = test['PUN_n']
    test_x = test.drop(['PUN_n'], axis=1)
    train_x = train_x.values
    test_x = test_x.values

    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = load_data()

The first step is to create a model which takes an argument hp which is the space of hyperparameters and returns a compiled model. The compilation step is where the optimizer along with the loss function and the metric are defined. 

In Keras Tuner, hyperparameters have a type (possibilities are Float, Int, Boolean, and Choice) and a unique name. Then, a set of options to help guide the search need to be set:

- a minimal, a maximal and a default value for the Float and the Int types
- a set of possible values for the Choice type
- optionally, a sampling method within linear, log or reversed log. Setting this parameter allows to add prior knowledge you might have about the tuned parameter. We'll see in the next section how it can be used to tune the learning rate for instance
- optionally, a step value, i.e the minimal step between two hyperparameter values

In [9]:
def create_model(hp):
    node1 = hp.Choice('node1', values=[29,  33,  35,  50, 104])
    node2 = hp.Choice('node2', values=[29,  33,  35,  50, 104])
    dropout1 = hp.Choice('dropout1', values=[0.34, 0.35, 0.42, 0.44, 0.79])
    dropout2 = hp.Choice('dropout2', values=[0.26, 0.48, 0.52, 0.71, 0.79])
    activation1 = hp.Choice('activation1', values=['tanh', 'softmax', 'sigmoid', 'relu'])
    activation2 = hp.Choice('activation2', values=['tanh', 'softmax', 'sigmoid', 'relu'])
    activation3 = hp.Choice('activation3', values=['tanh', 'softmax', 'sigmoid', 'relu'])

    def get_optimizer(optimizer, lr):
        if optimizer == 'adam':
            return tensorflow.keras.optimizers.Adam(learning_rate=lr)
        elif optimizer == 'rmsprop':
            return tensorflow.keras.optimizers.RMSprop(learning_rate=lr)
        elif optimizer == 'sgd':
            return tensorflow.keras.optimizers.SGD(learning_rate=lr)

    lr = hp.Choice('lr', values=[0.0011, 0.0331, 0.0464, 0.0654, 0.0926])
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])

    optimizer = get_optimizer(optimizer, lr)

    model = Sequential()
    model.add(Dense(node1, activation=activation1, input_shape=(train_x.shape[1],)))
    model.add(Dropout(rate=dropout1))
    model.add(Dense(node2, activation=activation2))
    model.add(Dropout(rate=dropout2))
    model.add(Dense(1, activation=activation3))


    model.compile(loss='mean_squared_error',
                  optimizer=optimizer,
                  metrics=['mae'])
    return model

Next, instantiate a tuner to determine which hyperparameter combinations should be tested. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial). Available tuners are RandomSearch and Hyperband. 

In [10]:
tuner = RandomSearch(create_model, executions_per_trial=3, objective=kerastuner.Objective("loss", direction="min"), 
                max_trials = 2, project_name='NN_gas')

Traceback (most recent call last):
  File "C:\Users\mpich\anaconda3\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-9-0d7f17a7051c>", line 21, in create_model
    optimizer = get_optimizer(optimizer, lr)
  File "<ipython-input-9-0d7f17a7051c>", line 12, in get_optimizer
    return tensorflow.keras.optimizers.Adam(learning_rate=lr)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 471, in __init__
    super(Adam, self).__init__(**kwargs)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 68, in __init__
    'passed to optimizer: ' + str(k))
TypeError: Unexpected keyword argument passed to optimizer: learning_rate


Traceback (most recent call last):
  File "C:\Users\mpich\anaconda3\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-9-0d7f17a7051c>", line 21, in create_model
    optimizer = get_optimizer(optimizer, lr)
  File "<ipython-input-9-0d7f17a7051c>", line 12, in get_optimizer
    return tensorflow.keras.optimizers.Adam(learning_rate=lr)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 471, in __init__
    super(Adam, self).__init__(**kwargs)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 68, in __init__
    'passed to optimizer: ' + str(k))
TypeError: Unexpected keyword argument passed to optimizer: learning_rate


Traceback (most recent call last):
  File "C:\Users\mpich\anaconda3\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-9-0d7f17a7051c>", line 21, in create_model
    optimizer = get_optimizer(optimizer, lr)
  File "<ipython-input-9-0d7f17a7051c>", line 12, in get_optimizer
    return tensorflow.keras.optimizers.Adam(learning_rate=lr)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 471, in __init__
    super(Adam, self).__init__(**kwargs)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 68, in __init__
    'passed to optimizer: ' + str(k))
TypeError: Unexpected keyword argument passed to optimizer: learning_rate


Traceback (most recent call last):
  File "C:\Users\mpich\anaconda3\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-9-0d7f17a7051c>", line 21, in create_model
    optimizer = get_optimizer(optimizer, lr)
  File "<ipython-input-9-0d7f17a7051c>", line 12, in get_optimizer
    return tensorflow.keras.optimizers.Adam(learning_rate=lr)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 471, in __init__
    super(Adam, self).__init__(**kwargs)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 68, in __init__
    'passed to optimizer: ' + str(k))
TypeError: Unexpected keyword argument passed to optimizer: learning_rate


Traceback (most recent call last):
  File "C:\Users\mpich\anaconda3\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-9-0d7f17a7051c>", line 21, in create_model
    optimizer = get_optimizer(optimizer, lr)
  File "<ipython-input-9-0d7f17a7051c>", line 12, in get_optimizer
    return tensorflow.keras.optimizers.Adam(learning_rate=lr)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 471, in __init__
    super(Adam, self).__init__(**kwargs)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 68, in __init__
    'passed to optimizer: ' + str(k))
TypeError: Unexpected keyword argument passed to optimizer: learning_rate


Traceback (most recent call last):
  File "C:\Users\mpich\anaconda3\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-9-0d7f17a7051c>", line 21, in create_model
    optimizer = get_optimizer(optimizer, lr)
  File "<ipython-input-9-0d7f17a7051c>", line 12, in get_optimizer
    return tensorflow.keras.optimizers.Adam(learning_rate=lr)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 471, in __init__
    super(Adam, self).__init__(**kwargs)
  File "C:\Users\mpich\anaconda3\lib\site-packages\tensorflow\python\keras\optimizers.py", line 68, in __init__
    'passed to optimizer: ' + str(k))
TypeError: Unexpected keyword argument passed to optimizer: learning_rate


RuntimeError: Too many failed attempts to build model.

There are three possible algorithms that can be used for the hyperparameters optimization: random searc, bayesian optimization adn Hyperband.
    
- Random search: randomly tries a combination of hyperparameters from a given search space.
  - max_trials variable: number of hyperparameter combinations that will be tested by the tuner
  - execution_per_trial variable: the number of models that should be built and fit for each trial for robustness purposes. 
- Hyperband is an optimized version of random search which uses early-stopping to speed up the hyperparameter tuning process. The main idea is to fit a large number of models for a small number of epochs and to only continue training for the models achieving the highest accuracy on the validation set. 
  - max_epochs variable is the max number of epochs that a model can be trained for.
- Bayesian optimization is a probabilistic model that maps the hyperparameters to a probability score on the objective function. Unlike Random Search and Hyperband models, Bayesian Optimization keeps track of its past evaluation results and uses it to build the probability model.

In [ ]:
tuner.search_space_summary()

Run the hyperparameters optimization algorithm (tuner) with the search method.

The search function takes as input the training data and a validation split to perform hyperparameter combinations evaluation. The epochs parameter is used in random search and Bayesian Optimization to define the number of training epochs for each hyperparameter combination.

In [ ]:
tuner.search(x=train_x, y=train_y, epochs=30, batch_size = 24,
             callbacks=[tensorflow.keras.callbacks.EarlyStopping('val_loss', patience=3)], 
             validation_data=(test_x, test_y))

In [ ]:
tuner.results_summary()

Print the best model(s) and get a summary of the results.

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

In [ ]:
loss, accuracy = best_model.evaluate(test_x, test_y)
print(loss)
print(accuracy)

In [ ]:
loss

In [ ]:
accuracy

In [ ]:
print(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)